In [3]:
""" 
g = -10 
GPS = [xt,yt,zt]
base station = [D1,D2,D3,D4] from z=10 
IMU = [xt,yt,zt,vxt,vyt,vzt] 
motion noise = N(0,R) ; R is diag(sigx^2,sigy^2,sigz^2,sigvx^2,sigvy^2,sigvz^2) 
sigx=sigy=sigz=0.01
sigvx=sigvy=sigvz=0.1 

measurement noise = N(0,Q) Q = sig^2I 
sigGPS = 0.1 
sigBS = 0.1 
sigIMU = 0.1 

ut is only g in zt direction 

Time in seconds 
N = 130 time steps , delta_time = 0.01 
field is 100 x 64 , 
goal post is 8 x 3 , at (4,50,0);(4,50,3);(-4,50,3);(-4,50,0); 

boal init = [24.0,4.0,0.0,−16.04,36.8,8.61]T , with sigma = 1e-4 I_6 
state Xt = [xt,yt,zt,vxt,vyt,vzt]T 
"""
import numpy as np 
import math 
from numpy.random import multivariate_normal  
import plotly.express as px
import plotly.graph_objects as go 
plot_dir = "plots_part_2/"
tol = 0

In [ ]:
g = -10 
def u(t):
    return np.array([g]).reshape(1,1) 
time_step = 0.01 
N = 130 
time = [i*time_step for i in range(N)]


In [ ]:
# Part (a) motion model 
sigGPS = 0.1 
sigBS = 0.1 
sigIMU = 0.1  
sigx=sigy=sigz=0.01 
sigvx=sigvy=sigvz=0.1

R = np.diag([sigx**2,sigy**2,sigz**2,sigvx**2,sigvy**2,sigvz**2])
Q_GPS = sigGPS**2*np.eye(3) 
Q_BS = sigBS**2*np.eye(4) 
Q_IMU = sigIMU**2*np.eye(6) 

mu_0 = np.transpose(np.array([24.0,4.0,0.0,-16.04,36.8,8.61]))
mu_0 = mu_0.reshape((6,1)) 
sigma_0 = 1e-4*np.eye(6) 

At = np.array([
    [1,0,0,time_step,0,0],
    [0,1,0,0,time_step,0],
    [0,0,1,0,0,time_step],
    [0,0,0,1,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,1]
    ])
Bt =  np.transpose(np.array([0,0,0.5*(time_step**2),0,0,time_step])) 


